## DC2 Retrieve Forced Src catalogs of visit-level forced photoemtry from coadd Object catalog.  
Michael Wood-Vasey

This Notebook shows how to retrieve the list of forced sources from a given tract, patch, filter.  
The forced-source dataIds contain sufficient information to look up the calexp IDs to retrieve postage stamps.

In [ ]:
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage

In [ ]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
butler = dafPersist.Butler(repo)

In [ ]:
tract = 4850
patch = '5,6'

filt = 'r'
partial_data_id = {'tract': tract, 'patch': patch, 'filter': filt}

In [ ]:
dataset_type = 'forced_src'
subset = butler.subset(datasetType=dataset_type, dataId=partial_data_id)
data_ids = [dr.dataId for dr in subset
           if butler.datasetExists(datasetType=dataset_type,
                                   dataId=dr.dataId)]

In [ ]:
print(data_ids[0:3])

In [ ]:
dId = {'tract': 4848, 'patch': '5,6', 'filter': 'r', 'visit': 193861, 'raft': '1,4', 'sensor': '0,0'}

calibrated_exposure = butler.get(datasetType='calexp', dataId=dId)
forced_src = butler.get(datasetType=dataset_type, dataId=dId)

In [ ]:
cat = forced_src.asAstropy()

In [ ]:
calib = calibrated_exposure.getCalib()
calib.setThrowOnNegativeFlux(False)

mag, mag_err = calib.getMagnitude(forced_src['base_PsfFlux_flux'],
                                  forced_src['base_PsfFlux_fluxSigma'])
cat['mag'] = mag
cat['mag_err'] = mag_err
cat['snr'] = np.abs(cat['base_PsfFlux_flux'])/cat['base_PsfFlux_fluxSigma']

In [ ]:
print(cat)

This is per tract, so 22,062 sources seems potentially reasonble.

The `id` is the same `id` as in the coadd "reference" catalog (the result of merged detections from the individ

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
plt.scatter(np.rad2deg(cat['coord_ra']),
            np.rad2deg(cat['coord_dec']))
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')

In [ ]:
plt.scatter(cat['mag'], cat['snr'])
plt.xlabel('%s mag' % dId['filter'])
plt.ylabel('S/N')

Looks like a reasonable set of magnitudes and uncertainties.

## From one image to many
Let's pick one of the Objects and go through all of the data IDs in that filter for the tract, patch.

In [ ]:
object_id = cat[0]['id']

In [ ]:
def get_all_images_for_object_id_from_data_ids(object_id, data_refs, dataset_type='forced_src'):
    """Take an Object ID and data_ref list and return matching data_refs that contain that object and exist
    
    Note:  This perhaps isn't quite the function you wanted.
    You may have wanted a function that just took an object_id and filter.
    But we'll get there.
    """
    matching_data_refs = []
    for data_ref in data_refs:
        # Shoudl add check to make sure it exists
        cat = data_ref.get(datasetType=dataset_type)
        if object_id in cat['id']:
            matching_data_refs.append(data_ref)

    return matching_data_refs

But the above wasn't quite what you likely wanted.  You probably wanted to start with a given ObjectID from the Object (coadd) catalog and then get all of the images that include that object.